In [ ]:
!pip install -q google-cloud-aiplatform pytest

import os
import json
import pytest
from typing import Dict
import vertexai
from vertexai.generative_models import GenerativeModel
from vertexai.evaluation import EvalTask, MetricPromptTemplateExamples
import pandas as pd
import datetime

PROJECT_ID = "qwiklabs-gcp-02-e6e6123d96ed"
vertexai.init(project=PROJECT_ID, location="us-central1")
model = GenerativeModel("gemini-2.0-flash-exp")

print("Setup complete")


In [ ]:
def classify_question(question):
    system_prompt = """
    You are a helpful AI Assistant. Your task is to classify user questions into one of the following categories:

    1. Employment
    2. General Information
    3. Emergency Services
    4. Tax Related

    If a specific category cannot be determined, assign the category: 'General Information'.

    Your response must include **only** the category name — no additional text.

    Example:
    Question: What is the ambulance contact number?
    Answer: Emergency Services
    """

    try:
        response = model.generate_content(f"{system_prompt}\n\nQuestion: {question}")
        category = response.text.strip()

        valid_categories = ["Employment", "General Information", "Emergency Services", "Tax Related"]
        for valid_cat in valid_categories:
            if valid_cat.lower() in category.lower():
                return valid_cat

        return "General Information"
    except:
        return "General Information"

In [ ]:
def generate_social_media_post(announcement_type, details, platform="all"):
    system_prompt = """
    You are a communications assistant for a government agency. Your task is to draft clear, concise, and polite social media posts for official public announcements.

    Instructions:
    1. Keep the message under 200 words.
    2. Use a calm, professional, and reassuring tone.
    3. Clearly state what the announcement is about, who it affects, and when it applies.
    4. Include basic safety tips or recommended actions if applicable.
    5. Use simple, accessible language.
    6. End with 2-4 relevant hashtags.
    7. Do not use emojis.

    Only return the post text with hashtags at the end.
    """

    prompt = f"""
    Create a social media post for {announcement_type}.
    Details: {json.dumps(details)}
    Platform: {platform}
    """

    try:
        response = model.generate_content(f"{system_prompt}\n\n{prompt}")
        return {
            "status": "success",
            "generated_content": {"posts": {"twitter": response.text.strip()}},
            "raw_response": response.text
        }
    except Exception as e:
        return {"status": "error", "error": str(e), "generated_content": None}


In [ ]:
print("Testing classification:")
result = classify_question("How do I apply for a city job?")
print(f"Result: {result}")

print("\nTesting social media generation:")
social_result = generate_social_media_post("holiday", {"holiday": "New Year", "office_status": "closed"})
if social_result["status"] == "success":
    print(f"Post: {social_result['generated_content']['posts']['twitter']}")

In [ ]:
import pytest

class TestQuestionClassifier:

    def test_employment_category(self):
        question = "How do I apply for a government job?"
        category = classify_question(question)
        assert category == "Employment"

    def test_emergency_services_category(self):
        question = "What is the ambulance contact number?"
        category = classify_question(question)
        assert category == "Emergency Services"

    def test_tax_related_category(self):
        question = "How do I file my income tax returns?"
        category = classify_question(question)
        assert category == "Tax Related"

    def test_general_info_category(self):
        question = "What are the office hours?"
        category = classify_question(question)
        assert category == "General Information"

    def test_default_to_general_information(self):
        question = "Can you help me with this?"
        category = classify_question(question)
        assert category == "General Information"

class TestSocialMediaGenerator:

    def test_holiday_announcement(self):
        details = {"holiday": "Memorial Day", "office_status": "closed"}
        result = generate_social_media_post("holiday", details)
        assert result["status"] == "success"
        assert "twitter" in result["generated_content"]["posts"]

    def test_error_handling(self):
        result = generate_social_media_post(None, None)
        assert "status" in result

# Run pytest tests
def run_tests():
    print("\nRunning pytest tests:")
    test_class = TestQuestionClassifier()
    test_class.test_employment_category()
    test_class.test_emergency_services_category()
    test_class.test_tax_related_category()
    test_class.test_general_info_category()
    test_class.test_default_to_general_information()

    social_test = TestSocialMediaGenerator()
    social_test.test_holiday_announcement()
    social_test.test_error_handling()

    print("All pytest tests passed!")

run_tests()


In [ ]:
social_media_system_prompt = """
You are a communications assistant for a government agency. Your task is to draft clear, concise, and polite social media posts for official public announcements.

Instructions:
1. Keep the message under 200 words.
2. Use a calm, professional, and reassuring tone.
3. Clearly state what the announcement is about, who it affects, and when it applies.
4. Include basic safety tips or recommended actions if applicable.
5. Use simple, accessible language.
6. End with 2-4 relevant hashtags.
7. Do not use emojis.

Only return the post text with hashtags at the end.
"""

posts_data = [
    {
        "prompt": "Flood warnings issued for Mumbai from July 5 to July 7. Residents advised to stay alert.",
        "post": "Flood warnings in Mumbai from July 5-7. Please stay alert and avoid travel unless necessary. Stay safe everyone! #MumbaiFloods #StaySafe"
    },
    {
        "prompt": "City hospitals offer free vaccination camps on August 1 and August 2.",
        "post": "Free vaccines at city hospitals on Aug 1 & 2. Don't miss it! Protect yourself and loved ones. #VaccinationDrive #HealthFirst"
    },
    {
        "prompt": "All markets in Hyderabad closed on September 10 for Ganesh Chaturthi.",
        "post": "Markets closed Sept 10 for Ganesh Chaturthi. Plan your shopping ahead. #GaneshChaturthi #Hyderabad"
    }
]

eval_dataset = pd.DataFrame([
    {
        "instruction": social_media_system_prompt,
        "context": f"announcement: {item['prompt']}",
        "response": item["post"],
    } for item in posts_data
])
run_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
gcs_output_uri = f"gs://{PROJECT_ID}-eval-results/social-media-eval-{run_timestamp}"

eval_task = EvalTask(
    dataset=eval_dataset,
    metrics=[
        MetricPromptTemplateExamples.Pointwise.GROUNDEDNESS,
        MetricPromptTemplateExamples.Pointwise.INSTRUCTION_FOLLOWING,
        MetricPromptTemplateExamples.Pointwise.SAFETY
    ],
    experiment="social-media-evaluation"
)

result = eval_task.evaluate(
    prompt_template="Instruction: {instruction}. Context: {context}. Response: {response}",
    experiment_run_name=f"social-media-eval-{run_timestamp}"
)

print("✅ Evaluation complete!")
print("🎉 All functions tested and evaluated successfully!")